# Experiment 4: Scattering Features + Linear Model




In [1]:
import sys
sys.path.append('../src')

from utils.reduce import reduce_pca
from utils.split import train_test_split, train_test_split_pytorch
from utils.CustomDataset import CustomDataset
from numpy.random import RandomState
from itertools import product
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.optim
import pickle
import pandas as pd
import numpy as np
import mlflow
import matplotlib.pyplot as plt

In [2]:
#mlflow.set_experiment('scattering_linear_experiment')

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Upload Scattering Features

In [3]:
with open('../data/03_features/scatter_dict_tensor.pickle', 'rb') as handle:
    scatter_dict= pickle.load(handle)
    scattering_features= scatter_dict['features']
    labels = scatter_dict['label']
    pids=  scatter_dict['pid']
    scattering_params = { 'J':scatter_dict['J'],
                         'M':scatter_dict['M'],
                         'N':scatter_dict['N']}

In [4]:
###############################################################################
# If a GPU is available, let's use it!
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

###############################################################################
###############################################################################
# For reproducibility, we fix the seed of the random number generator.
torch.manual_seed(42)
prng = RandomState(42)
################################################################################

# Create dataloader

In [5]:
#number of coefficients 
K = scattering_features.shape[1]
H = scattering_features.shape[2]
W = scattering_features.shape[3]

In [6]:
train_set, train_set_labels, test_set,  test_set_labels= train_test_split_pytorch(scattering_features, pids, labels)
train_loader = CustomDataset(train_set, train_set_labels)
test_loader = CustomDataset(test_set,  test_set_labels)

In [7]:
if use_cuda:
    num_workers = 4
    pin_memory = True  #Pinned memory is useful to issue host-device memory transfer at maximum rates 
else:
    num_workers = 0
    pin_memory = False

In [8]:
#train_loader = torch.utils.data.DataLoader(train_loader, batch_size=10, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
#test_loader = torch.utils.data.DataLoader(test_loader, batch_size=10, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)

# Define Traning and Testing Functions

In [41]:
def train(model, device, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target.type(torch.int64))  
        loss.backward()
        optimizer.step()


        

def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.max(1, keepdim = True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    return 100. * correct / test_loader.__len__()

# Define Network

In [42]:
class View(nn.Module):
    def __init__(self, *args):
        super(View, self).__init__()
        self.shape = args

    def forward(self, x):
        return x.view(-1,*self.shape)   # reshape of size (-1,*self.shape)

In [50]:
model = nn.Sequential(
    View(K, H, W),
    nn.BatchNorm2d(K),
    View(K * H * W),
    nn.Linear(K * H * W, 2)).to(device)

model = nn.Sequential(
    View(K, scattering_params, 7),
    nn.BatchNorm2d(K),
    View(K * 7 * 7),
    nn.Linear(K * 7 * 7, 10)).to(device)

# Training and Testing

In [51]:
# Optimizer
# TO DO SHUFFLE
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9,
                            weight_decay=0.0005)
for epoch in range(0, 20):
    train( model, device, train_loader, optimizer)

acc = test(model, device, test_loader)
print('Scattering order 2 linear model test accuracy: %.2f' % acc)


Scattering order 2 linear model test accuracy: 33.33


# Run Mlflow to see results

`!mlflow ui`

Should launch something like this:



In [ ]:
# !mlflow ui
